In [34]:
from utilities import *
from multimod_graph import *

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_icu = pd.read_csv('/drives/drive1/Patient Safety Graphs/data_release_12132022/ICU_STAY_20221128.csv', sep=',', encoding="utf-8")
df_comp_poa = pd.read_csv('/drives/drive1/Patient Safety Graphs/data_release_12132022/COMP10_POA_20221128.csv', sep=',')

In [ ]:
#some location has multiple type, so we just combine them
df_icu['ON_HOSP_ORG_LVL4_DESC'] = df_icu['ON_HOSP_ORG_LVL4_DESC'] + "_" + df_icu['ON_BED_TYPE']

In [ ]:
df_feats_clinical=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_clinical.pkl','rb'))
df_master_preprocessed=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/intermediary/df_master_preprocessed.pkl','rb'))
df_provider_preprocessed=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/intermediary/df_provider_preprocessed.pkl','rb'))

In [4]:
df_master_index_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_all.pkl','rb'))

df_master_index_train=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_train.pkl','rb'))
df_master_index_val=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_val.pkl','rb'))
df_master_index_test=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_test.pkl','rb'))

In [5]:
df_master_index_train_val=pd.concat([df_master_index_train,df_master_index_val])

In [6]:
df_master_index_train_val.shape[0]-df_master_index_train.shape[0]-df_master_index_val.shape[0]

0

In [ ]:
df_provider_preprocessed_train = df_provider_preprocessed.loc[df_provider_preprocessed['CASE_NUMBER'].isin(df_master_index_train['CASE_NUMBER'])].reset_index(drop=True)
df_provider_preprocessed_train_val = df_provider_preprocessed.loc[df_provider_preprocessed['CASE_NUMBER'].isin(df_master_index_train_val['CASE_NUMBER'])].reset_index(drop=True)

In [ ]:
df_provider_preprocessed_train.shape,df_provider_preprocessed_train_val.shape

In [7]:
hospenc_index_test = df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'].drop_duplicates().values
hospenc_index_val = df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'].drop_duplicates().values

hospenc_index_all = df_master_index_all['ENCRYPTED_HOSP_ENCOUNTER'].drop_duplicates().values
hospenc_index_train = df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'].drop_duplicates().values
hospenc_index_train_val = df_master_index_train_val['ENCRYPTED_HOSP_ENCOUNTER'].drop_duplicates().values

In [8]:
print(len(hospenc_index_all))
print(len(set(hospenc_index_all)))

136647
136647


# Multimodal Graph 1: get features using the entire data

In [ ]:
G_multimodal_all, node_feat_icd, node_feat_icu, node_feats_provider = multimod_graph(df_master_preprocessed,\
                                                                                       df_provider_preprocessed,\
                                                                                       df_comp_poa,\
                                                                                       df_icu,\
                                                                                       hospenc_index_all)

In [ ]:
pickle.dump(G_multimodal_all,open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/intermediary/G_multimodal_all.pkl','wb'))

In [ ]:
pickle.dump(node_feat_icd,open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/node_feats_icd.pkl','wb'))
pickle.dump(node_feat_icu,open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/node_feats_icu.pkl','wb'))
pickle.dump(node_feats_provider,open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/node_feats_provider.pkl','wb'))

In [3]:
G_multimodal_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/intermediary/G_multimodal_all.pkl','rb'))

In [9]:
multimodal_graph_edgelist = nx.to_pandas_edgelist(G_multimodal_all)

In [10]:
multimodal_graph_edgelist.head(2)

,source,target,weight
0,"b""\x16\x7f\x17:\xd6\x7f\xb4Y\tJ\xc6\x1f\x8d'\x8b""",UF NTOR_ACUTE,3
1,"b""\x16\x7f\x17:\xd6\x7f\xb4Y\tJ\xc6\x1f\x8d'\x8b""",UF UA4W_ACUTE,3


In [11]:
nodes_from_list1 = list(set(multimodal_graph_edgelist['source']))
nodes_from_list2 = list(set(multimodal_graph_edgelist['target']))

In [12]:
nodes_from_list = np.concatenate([nodes_from_list1,nodes_from_list2],axis=0)

In [13]:
nodes_graph = np.array(G_multimodal_all.nodes)

In [14]:
graph = Graph(multimodal_graph_edgelist.values,directed=False,weighted=True)

Reading graph: 100%|██████████████| 2807644/2807644 [00:04<00:00, 616103.02it/s]


In [15]:
np.random.seed(seed=666)
n2v = Node2Vec(graph,dim=128,walk_length=20,context=10,workers=32)

In [16]:
n2v.train(epochs=30)

Training: 100%|█████████████████████| 4320990/4320990 [08:34<00:00, 8398.63it/s]


In [17]:
emb_df = (
    pd.DataFrame(
        [n2v.wv[str(n)] for n in hospenc_index_all],
        index = hospenc_index_all
    )
)
emb_df = emb_df.reset_index()
emb_df.columns=np.concatenate((['ENCRYPTED_HOSP_ENCOUNTER'],['Node2Vec_'+str(n) for n in emb_df.columns[1:]]),axis=0)


In [18]:
emb_df.shape

(136647, 129)

In [19]:
feats_n2v_all_train = emb_df[emb_df['ENCRYPTED_HOSP_ENCOUNTER'].isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])]
feats_n2v_all_val= emb_df[emb_df['ENCRYPTED_HOSP_ENCOUNTER'].isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])]
feats_n2v_all_test= emb_df[emb_df['ENCRYPTED_HOSP_ENCOUNTER'].isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])]

In [20]:
pickle.dump(feats_n2v_all_train,open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_train.pkl','wb'))
pickle.dump(feats_n2v_all_val,open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_val.pkl','wb'))
pickle.dump(feats_n2v_all_test,open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_test.pkl','wb'))

In [ ]:
multimodal_all_adj_mat_HospecnProv = 
multimodal_all_adj_mat_HospecnICU = 
multimodal_all_adj_mat_HospecnICD = 

In [ ]:
pickle.dump(G_multimodal_all
pickle.dump(G_multimodal_all
pickle.dump(G_multimodal_all

# MUltimodal Graph 2: get features from train:train, val:val+train, test: entire

In [21]:
G_multimodal_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/intermediary/G_multimodal_all.pkl','rb'))
G_multimodal_train_val = G_multimodal_all
G_multimodal_train_val.remove_nodes_from(hospenc_index_test)

In [22]:
multimodal_graph_edgelist_train_val = nx.to_pandas_edgelist(G_multimodal_train_val)
nodes_from_list1 = list(set(multimodal_graph_edgelist_train_val['source']))
nodes_from_list2 = list(set(multimodal_graph_edgelist_train_val['target']))
nodes_from_list_train_val = np.concatenate([nodes_from_list1,nodes_from_list2],axis=0)

In [23]:
is_in_set_pnb(hospenc_index_val,nodes_from_list_train_val).mean()

1.0

In [24]:
np.random.seed(seed=666)

graph_train_val = Graph(multimodal_graph_edgelist_train_val.values,directed=False,weighted=True)
n2v_train_val = Node2Vec(graph_train_val,dim=128,walk_length=20,context=10,workers=32)
n2v_train_val.train(epochs=30)

Training: 100%|█████████████████████| 3896880/3896880 [07:48<00:00, 8325.49it/s]


In [25]:
emb_df_train_val = (
    pd.DataFrame(
        [n2v_train_val.wv[str(n)] for n in hospenc_index_train_val],
        index = hospenc_index_train_val
    )
)
emb_df_train_val = emb_df_train_val.reset_index()
emb_df_train_val.columns=np.concatenate((['ENCRYPTED_HOSP_ENCOUNTER'],['Node2Vec_'+str(n) for n in emb_df_train_val.columns[1:]]),axis=0)
feats_n2v_val_separate= emb_df_train_val[emb_df_train_val['ENCRYPTED_HOSP_ENCOUNTER'].isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])]
pickle.dump(feats_n2v_val_separate,open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_val_separate.pkl','wb'))

In [26]:
G_multimodal_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/intermediary/G_multimodal_all.pkl','rb'))
G_multimodal_train = G_multimodal_all
G_multimodal_train.remove_nodes_from(hospenc_index_test)
G_multimodal_train.remove_nodes_from(hospenc_index_val)

In [27]:

multimodal_graph_edgelist_train = nx.to_pandas_edgelist(G_multimodal_train)
nodes_from_list1 = list(set(multimodal_graph_edgelist_train['source']))
nodes_from_list2 = list(set(multimodal_graph_edgelist_train['target']))
nodes_from_list_train = np.concatenate([nodes_from_list1,nodes_from_list2],axis=0)

In [28]:
is_in_set_pnb(hospenc_index_train,nodes_from_list_train).sum()

109318

In [29]:
np.random.seed(seed=666)

graph_train = Graph(multimodal_graph_edgelist_train.values,directed=False,weighted=True)
n2v_train= Node2Vec(graph_train,dim=128,walk_length=20,context=10,workers=32)
n2v_train.train(epochs=30)

Training: 100%|█████████████████████| 3472770/3472770 [07:37<00:00, 7597.53it/s]


In [30]:
emb_df_train = (
    pd.DataFrame(
        [n2v_train.wv[str(n)] for n in hospenc_index_train],
        index = hospenc_index_train
    )
)
emb_df_train = emb_df_train.reset_index()
emb_df_train.columns=np.concatenate((['ENCRYPTED_HOSP_ENCOUNTER'],['Node2Vec_'+str(n) for n in emb_df_train.columns[1:]]),axis=0)
feats_n2v_train_separate= emb_df_train[emb_df_train['ENCRYPTED_HOSP_ENCOUNTER'].isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])]
pickle.dump(feats_n2v_train_separate,open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_train_separate.pkl','wb'))
